# D11 Pivot tables và cross-tabulation

## Mục đích

Làm quen với các cách thống kê bảng biểu trong Pandas.


## Pivot tables

Nếu bạn thường xuyên sử dụng MS Excel thì chắc cũng biết tính năng nổi tiếng Pivot table, cho phép chúng ta lập các bảng số liệu tổng kết. Trong Pandas, bạn cũng có thể làm tương tự như vậy. Trong ví dụ dưới đây, chúng ta sẽ thống kê biến IRP 4s theo giới tính (cột) và nhóm BMI (hàng).

In [1]:
import pandas as pd

d = pd.read_excel("hrm.xlsx", index_col="id")

# Tạo các biến cho BMI và xóa bản ghi NA của BMI và IRP4s
d["bmi"] = d["weight"] / d["height"] ** 2
d = d.dropna(subset=["bmi", "les_irp4s"])
d["bmi_group"] = (1 + d["bmi"].ge(18.5) + d["bmi"].gt(23)).replace({1: "Thiếu cân", 2: "Bình thường", 3: "Thừa cân"})
d["sex"] = d["sex"].replace({0: "Nữ", 1: "Nam"})

Đầu tiên, chúng ta sẽ đếm số bản ghi có trong mỗi nhóm.

In [2]:
d.pivot_table(values="les_irp4s", index="bmi_group", columns="sex", aggfunc="count")

sex,Nam,Nữ
bmi_group,,
Bình thường,52,132
Thiếu cân,7,17
Thừa cân,46,57


Chúng ta có thể tính trung vị của IRP 4s trong mỗi nhóm.

In [3]:
d.pivot_table(values="les_irp4s", index="bmi_group", columns="sex", aggfunc="median")

sex,Nam,Nữ
bmi_group,,
Bình thường,5.1,5.65
Thiếu cân,6.1,6.30
Thừa cân,3.2,4.50


Chúng ta có thể cùng một lúc thống kê nhiều chỉ số khác nhau, ví dụ: N, median, và Q1-Q3. Để tạo ra các giá trị Q1-Q3, chúng ta sẽ cần dùng hàm lambda vì phải đưa thêm đối số phân vị cho hàm này.

In [4]:
d.pivot_table(values="les_irp4s", index="bmi_group", columns="sex",
    aggfunc=[
        "count",
        "median",
        lambda x: (x.quantile(0.25), x.quantile(0.25))
    ])

count      median              <lambda>            
sex           Nam   Nữ    Nam    Nữ             Nam          Nữ
bmi_group                                                      
Bình thường    52  132    5.1  5.65  (2.475, 2.475)  (3.5, 3.5)
Thiếu cân       7   17    6.1  6.30    (2.95, 2.95)  (2.6, 2.6)
Thừa cân       46   57    3.2  4.50  (1.525, 1.525)  (2.7, 2.7)

Điểm đặc biệt của pivot table là bạn có thể tham các cột / hàng tổng.

In [5]:
pt = d.pivot_table(values="les_irp4s", index="bmi_group", columns="sex",
    aggfunc=[
        "count",
        "median",
        lambda x: (x.quantile(0.25), x.quantile(0.75))
    ],
    margins=True, margins_name="Tất cả")
pt

count             median                    <lambda>  \
sex           Nam   Nữ Tất cả    Nam    Nữ Tất cả            Nam   
bmi_group                                                          
Bình thường    52  132    184    5.1  5.65   5.55  (2.475, 7.55)   
Thiếu cân       7   17     24    6.1  6.30   6.20   (2.95, 6.55)   
Thừa cân       46   57    103    3.2  4.50   3.80  (1.525, 5.35)   
Tất cả        105  206    311    4.0  5.35   4.90     (1.7, 6.5)   

                                                      
sex                    Nữ                     Tất cả  
bmi_group                                             
Bình thường  (3.5, 7.225)               (3.075, 7.3)  
Thiếu cân      (2.6, 9.0)               (2.55, 8.45)  
Thừa cân       (2.7, 7.3)  (1.95, 6.699999999999999)  
Tất cả       (3.0, 7.375)                (2.5, 7.25)

Một nhược điểm của pivot table là tầng tên hàm (`"count"`, `"median"`, `"<lambda>"`) nằm trên cùng. Chúng ta sẽ cần sắp xếp lại thứ tự các cột này. Cách làm đơn giản nhất là unstack và stack tầng tên hàm.

In [6]:
pt.stack(level=0).unstack(level=1)

sex                    Nam                         Nữ               \
                  <lambda> count median      <lambda> count median   
bmi_group                                                            
Bình thường  (2.475, 7.55)    52    5.1  (3.5, 7.225)   132   5.65   
Thiếu cân     (2.95, 6.55)     7    6.1    (2.6, 9.0)    17    6.3   
Thừa cân     (1.525, 5.35)    46    3.2    (2.7, 7.3)    57    4.5   
Tất cả          (1.7, 6.5)   105    4.0  (3.0, 7.375)   206   5.35   

sex                             Tất cả               
                              <lambda> count median  
bmi_group                                            
Bình thường               (3.075, 7.3)   184   5.55  
Thiếu cân                 (2.55, 8.45)    24    6.2  
Thừa cân     (1.95, 6.699999999999999)   103    3.8  
Tất cả                     (2.5, 7.25)   311    4.9

Tuy nhiên, cách làm này sẽ xáo trộn thứ tự các chỉ số thống kê. Lí do là vì khi stack, Pandas tự động sắp xếp lại các tên theo thứ tự tăng dần. Một giải pháp cho việc này là đổi tên các cột trước khi stack. Bạn chú ý trong hàm `rename()` mình thêm đối số `level` để chỉ đổi tên ở trong tầng tên hàm.

In [7]:
pt.rename(columns={"count": "1_N", "median": "2_Median", "<lambda>": "3_Q1-Q3"}, level=0) \
    .stack(level=0)

sex                             Nam            Nữ                     Tất cả
bmi_group                                                                   
Bình thường 1_N                  52           132                        184
            2_Median            5.1          5.65                       5.55
            3_Q1-Q3   (2.475, 7.55)  (3.5, 7.225)               (3.075, 7.3)
Thiếu cân   1_N                   7            17                         24
            2_Median            6.1           6.3                        6.2
            3_Q1-Q3    (2.95, 6.55)    (2.6, 9.0)               (2.55, 8.45)
Thừa cân    1_N                  46            57                        103
            2_Median            3.2           4.5                        3.8
            3_Q1-Q3   (1.525, 5.35)    (2.7, 7.3)  (1.95, 6.699999999999999)
Tất cả      1_N                 105           206                        311
            2_Median            4.0          5.35                        4.9
            3_Q1-Q3      (1.7, 6.5)  (3.0, 7.375)                (2.5, 7.25)

Sau đó, chúng ta đổi lại tên cột khi đã unstack xong. Mình cũng đổi luôn tên các axis để bảng trông hoàn thiện.

In [8]:
pt.rename(columns={"count": "1_N", "median": "2_Median", "<lambda>": "3_Q1-Q3"}, level=0) \
    .stack(level=0).unstack(level=1) \
    .rename(columns=lambda x: x[2:], level=1) \
    .rename_axis(index="BMI", columns=["Giới", ""])

Giới         Nam                         Nữ                      Tất cả  \
               N Median          Q1-Q3    N Median         Q1-Q3      N   
BMI                                                                       
Bình thường   52    5.1  (2.475, 7.55)  132   5.65  (3.5, 7.225)    184   
Thiếu cân      7    6.1   (2.95, 6.55)   17    6.3    (2.6, 9.0)     24   
Thừa cân      46    3.2  (1.525, 5.35)   57    4.5    (2.7, 7.3)    103   
Tất cả       105    4.0     (1.7, 6.5)  206   5.35  (3.0, 7.375)    311   

Giới                                           
            Median                      Q1-Q3  
BMI                                            
Bình thường   5.55               (3.075, 7.3)  
Thiếu cân      6.2               (2.55, 8.45)  
Thừa cân       3.8  (1.95, 6.699999999999999)  
Tất cả         4.9                (2.5, 7.25)

## Cross-tabulation

Nếu chỉ cần đếm và thống kê tỉ lệ phần trăm, bạn có thể dùng cross tabulation với hàm `pandas.crosstab()`.

In [9]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"])

bmi_group,Bình thường,Thiếu cân,Thừa cân
sex,,,
Nam,52,7,46
Nữ,132,17,57


Tương tự như pivot table, chúng ta có thể thêm các hàng và cột Tổng.

In [10]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"], margins=True)

bmi_group,Bình thường,Thiếu cân,Thừa cân,All
sex,,,,
Nam,52,7,46,105
Nữ,132,17,57,206
All,184,24,103,311


Để tính tỉ lệ, bạn sẽ cần thiết lập tỉ lệ này được tính theo hàng, cột, hay cho toàn bộ bảng. Hãy xem sự khác nhau.

In [11]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"], margins=True, normalize="index")

bmi_group,Bình thường,Thiếu cân,Thừa cân
sex,,,
Nam,0.495238,0.066667,0.438095
Nữ,0.640777,0.082524,0.276699
All,0.591640,0.077170,0.331190


In [12]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"], margins=True, normalize="columns")

bmi_group,Bình thường,Thiếu cân,Thừa cân,All
sex,,,,
Nam,0.282609,0.291667,0.446602,0.337621
Nữ,0.717391,0.708333,0.553398,0.662379


In [13]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"], margins=True, normalize="all")

bmi_group,Bình thường,Thiếu cân,Thừa cân,All
sex,,,,
Nam,0.167203,0.022508,0.14791,0.337621
Nữ,0.424437,0.054662,0.18328,0.662379
All,0.591640,0.077170,0.33119,1.000000


Cross tabulation cũng có thể được sử dụng tương tự pivot table để tính các chỉ số tổng hợp khác. Để làm việc này thì mình sẽ "ưa chuộng" sử dụng pivot table hơn.

In [14]:
pd.crosstab(index=d["sex"], columns=d["bmi_group"], values=d["les_irp4s"],
    aggfunc=["mean", "std"], margins=True)

mean                                       std            \
bmi_group Bình thường Thiếu cân  Thừa cân       All Bình thường Thiếu cân   
sex                                                                         
Nam          5.319231  4.071429  3.839130  4.587619    4.465337  4.452233   
Nữ           6.133333  7.500000  5.387719  6.039806    4.580621  5.662486   
All          5.903261  6.500000  4.696117  5.549518    4.551033  5.478178   

                               
bmi_group  Thừa cân       All  
sex                            
Nam        4.051473  4.307813  
Nữ         3.912438  4.515833  
All        4.030397  4.492696

---

[Bài trước](./10_reshape.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()